In [1]:
import re
from Bio import SeqIO
# from StringIO import StringIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import numpy as np
import itertools
import pandas as pd

## Question No.1

In [2]:
def get_ind(indfile):
    with open(indfile) as topo_file:
        lists = []
        for line in topo_file:
            lists += list(np.array(list(line.rstrip('\n').split(',')), dtype=np.int32))
        return(lists)
get_ind('introns.txt')

[2, 12, 15, 19]

In [3]:
def ExcludeIntrons(gene, introns):
    inds = get_ind(introns)
    with open(gene) as gene_file:
        gene_seq = (gene_file.read())
        new_gene_seq = gene_seq
#         print('The Original gene is:', gene_seq)
        for i in range(int(len(inds)/2)):
            intron = str(gene_seq[inds[2*i]:inds[2*i+1]+1])
#             print('The intron of start: {} and stop {} is {}:'.format(inds[2*i],inds[2*i+1],intron))
            new_gene_seq = new_gene_seq.replace(intron, '')
#         print('The gene after introns removal is:', new_gene_seq)
    return new_gene_seq

new_gene = ExcludeIntrons('genomic_dna.txt', 'introns.txt')
print(new_gene)

GAAT


## Question No.2

In [4]:
def getFragmentSize(seq):
    cut = 'GAATTC'
    index = seq.find(cut)
    fragment1 = seq[:index+1]
    fragment2 = seq[index+1:]
#     print(fragment1, len(fragment1))
#     print(fragment2, len(fragment2))
#     print(len(seq), len(fragment1) + len(fragment2))
    return len(fragment1) , len(fragment2)
seq = 'ACTGATCGATTACGTATAGTAGAATTCTATCATACATATATATCGATGCGTTCAT'
frag1, frag2 = getFragmentSize(seq)
print(frag1, frag2)    

22 33


## Question No.3

In [5]:
data = pd.read_csv('data.csv')
# adding column name to the respective columns
data.columns =['Species_name', 'Sequence', 'Gene_name', 'Expression_level']
print(len(data))
print(data.head())

5
              Species_name                                           Sequence  \
0  Drosophila melanogaster  actgtgacgtgtactgtacgactatcgatacgtagtactgatcgct...   
1      Drosophila simulans  atcgatcatgtcgatcgatgatgcatccgactatcgtcgatcgtga...   
2        Drosophila yakuba         cgcgcgctcgcgcatacggcctaatgcgcgcgctagcgatgc   
3     Drosophila ananassae  ttacgatcgatcgatcgatcgatcgtcgatcgtcgatgctacatcg...   
4     Drosophila ananassae  gcatcgatcgatcgcggcgcatcgatcgcgatcatcgatcatacgc...   

  Gene_name  Expression_level  
0    jdg766               185  
1    kdy533               485  
2    hdt739                85  
3    hdu045               356  
4    teg436               222  


### 1. A function to print the genes names of a specific Species
- Takes the Species name as an input argument
- Prints the names of genes belonging to this species 
- Retuens a list of these genes names

In [6]:
def specFinder(spec):
    genes_names = []
    for i in range(len(data)):
        if data.loc[i,'Species_name'] == spec:
            print(data.loc[i,'Gene_name'])
            genes_names.append(data.loc[i,'Gene_name'])
    return genes_names

In [7]:
spec = 'Drosophila melanogaster'
names = specFinder(spec)

jdg766


In [8]:
spec = 'Drosophila simulans'
names = specFinder(spec)

kdy533


### 2. A function to print the genes names of a specific length (90:110)
- Takes x and y lenghts boundries as input argument
- Prints the names of genes of sequence lenght within these boundries 
- Retuens a list of these genes names

In [9]:
def lenFinder(x, y):
    genes_names = []
    for i in range(len(data)):
        seq = data.loc[i,'Sequence']
        if (x <= (len(seq)) <= y) :
            print(data.loc[i,'Gene_name'])
            genes_names.append(data.loc[i,'Gene_name'])
    if len(genes_names) == 0 : print('No genes within these boundris')
    return genes_names
names = lenFinder(90,110)

kdy533
teg436


### 3.  genes with AT content less than 0.5 and expression level greater than 200

In [10]:
# Content counter
def contentCounter(content, seq):
    seq = str(seq).lower()
    content = str(content).lower()
    count = seq.count(content)
    return 2*count/len(seq)

for i in range(len(data)):
    seq = data.loc[i,'Sequence']
    print(contentCounter('AT', data.loc[i,'Sequence']))


0.08974358974358974
0.2222222222222222
0.07142857142857142
0.1794871794871795
0.14285714285714285


In [11]:
for i in range(len(data)):
    seq = data.loc[i,'Sequence']
    ex_lvl = data.loc[i,'Expression_level']
    if (ex_lvl>200) and (contentCounter('AT', seq)<0.5): 
        print(data.loc[i,'Gene_name'])

kdy533
hdu045
teg436


### 4.Genes whose name begins with "k" or "h" except those belonging to Drosophila melanogaster.

In [12]:
for i in range(len(data)):
    if data.loc[i,'Species_name'] != 'Drosophila melanogaster':
        fst_lett = str(data.loc[i,'Gene_name'])[0]
        if (fst_lett == 'k') or (fst_lett == 'h'):
            print(data.loc[i,'Gene_name'])

kdy533
hdt739
hdu045


### 5. Massage for every AT content level

In [13]:
for i in range(len(data)):
    name = data.loc[i,'Gene_name']
    seq = data.loc[i,'Sequence']
    content = contentCounter('AT', seq)
    if 0.45<=content<=0.65:
        print('The gene of name ({}) has a medium AT content of ({})'.format(name, round(content,3)))
    elif content>0.65:
        print('The gene of name {} has a High AT content of {}'.format(name, round(content,3)))
    elif 0.45>content:
        print('The gene of name {} has a Low AT content of {}'.format(name, round(content,3)))

The gene of name jdg766 has a Low AT content of 0.09
The gene of name kdy533 has a Low AT content of 0.222
The gene of name hdt739 has a Low AT content of 0.071
The gene of name hdu045 has a Low AT content of 0.179
The gene of name teg436 has a Low AT content of 0.143
